<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


# Вызов функций с помощью помощников

Бывает, что пользовательская функциональность не может быть выполнена LLM Assistant, но LLM Assistant достаточно "умный", чтобы знать, как заполнить функцию Python для этой задачи. В этом случае мы можем предоставить пользовательскую функцию, которая будет выполняться на нашей стороне и работать совместно с помощником. Это мощный инструмент, потому что теперь у вас есть мощь OpenAI LLM, но с полной гибкостью любого кода Python, который вы можете запустить на своей стороне. Давайте разберем пример:

## Шаг 1: Определите свою пользовательскую функцию

In [1]:
def word_definition_quiz(word,definition_options):
    '''
    INPUTS: 
        word str = Одиночная строка, представляющая слово
        definition_options list[str] = Список из 4 потенциальных определений, из которых только одно является правильным!
    OUTPUTS:
        ответ str = Выбранное пользователем определение
    '''
    print("Привет! Давайте проверим ваши знания о словах.")
    print(f"Какое правильное определение этого слова {word}\n")
    
    for num,option in enumerate(definition_options):
        print('\n')
        print(f"Определение #{num} это: {option}")
    
    print('\n')
    num_choice = input("Каков ваш выбор? (Верните вариант с одним числом.)")
    
    return definition_options[int(num_choice)]

**Test the function.**

In [2]:
word = "серендипность"
definition_options = [
    "Случайное появление событий счастливым или благоприятным образом.",
    "Чувство сильного страха, шока или отвращения",
    "Метод живописи с использованием непрозрачных пигментов, размолотых в воде и загущенных клеящим веществом",
    "Действие, связанное с задержкой или откладыванием чего-либо."
]

# Вызовите функцию с этими аргументами
response = word_definition_quiz(word, definition_options)


Привет! Давайте проверим ваши знания о словах.
Какое правильное определение этого слова серендипность



Определение #0 это: Случайное появление событий счастливым или благоприятным образом.


Определение #1 это: Чувство сильного страха, шока или отвращения


Определение #2 это: Метод живописи с использованием непрозрачных пигментов, размолотых в воде и загущенных клеящим веществом


Определение #3 это: Действие, связанное с задержкой или откладыванием чего-либо.




In [3]:
response

'Чувство сильного страха, шока или отвращения'

## Шаг 2: Определите пользовательскую функцию в формате JSON.

Вот пример формата, в котором должна быть определена функция Python, наиболее актуальную информацию можно найти в документации OpenAI: https://platform.openai.com/docs/assistants/tools/function-calling.


**ОБЩИЙ ФОРМАТ ОПРЕДЕЛЕНИЯ ИНСТРУМЕНТОВ В ФОРМАТЕ JSON:**

    tool = {'type':'function',
                'function':{
                    'name': 'my_function_name',
                    'parameters':{
                        "type":"object",
                          "properties":{
                              "parameter_one": {'type':'string','description':"A text description for the LLM of what this parameter should be. Note the JSON type."},
                              "parameter_two":{'type':'integer','description':"A text description for the LLM of this parameter, note the JSON type."}
                          },
                        'required' : ['parameter_one','parameter_two']
                    }

        }
    } 

In [6]:
tool = {'type':'function',
            'function':{
                'name': 'my_function_name',
                'parameters':{
                    "type":"object",
                      "properties":{
                          "parameter_one": {'type':'string','description':"A text description for the LLM of what this parameter should be. Note the JSON type."},
                          "parameter_two":{'type':'integer','description':"A text description for the LLM of this parameter, note the JSON type."}
                      },
                    'required' : ['parameter_one','parameter_two']
                }

    }
} 

In [4]:
# https://community.openai.com/t/function-calling-passing-a-list-of-values/369032
function_json = {'type':'function',
            'function':{
                'name': 'word_definition_quiz',
                'parameters':{
                    "type":"object",
                    "properties":{
                          "word": {'type':'string','description':"Одно слово"},
                          "definitions_list":{'type':'array',
                                        'items':{'type':'string'},
                                        'description':"Python list строк определений, в котором только одно правильное определение относится к параметру 'word'. Только string определения."}
                      },
                    'required' : ["word","definitions_list"]
                }

    }
} 

## Шаг 3: Создайте помощника с помощью инструмента вызова функций

In [5]:
from openai import OpenAI
client = OpenAI()

In [7]:
assistant = client.beta.assistants.update(
  assistant_id="asst_AeWhKBH9uqoaHjJkVasyNITT",
  name="Бот для теста функций ",
  temperature=0,
  instructions="Вы помогаете создать тест для проверки определений слов, предоставляя слово, а затем несколько вариантов определений, где только один вариант является правильным.",
  model="gpt-3.5-turbo-0125",
  tools=[function_json]
)

## Шаг 4: Создайте поток, сообщение и запуск

In [8]:
thread = client.beta.threads.create()

In [30]:
thread_messages = client.beta.threads.messages.list(thread_id=thread.id)
print(thread_messages)


SyncCursorPage[Message](data=[Message(id='msg_bsKbGAITaY8EL34xQG8IhAKV', assistant_id='asst_AeWhKBH9uqoaHjJkVasyNITT', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Ответ неверный. Правильное определение слова "abate" - "to reduce in amount, degree, or intensity".'), type='text')], created_at=1715976469, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_MQvkJtkHdsfwa2N0Lv6SzScK', status=None, thread_id='thread_vZdRkNxIYvP2kJxSbqKTgRIH'), Message(id='msg_fkv7z3ECQZSYB1CQE69VYTgE', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Создайте новый список слов и определений вопроса викторины. Затем сообщите мне, является ли полученный ответ правильным.'), type='text')], created_at=1715975844, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_

In [12]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Создайте новый список слов и определений вопроса викторины. Затем сообщите мне, является ли полученный ответ правильным.",
)

In [14]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  tools=[function_json]
  )

In [15]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        print(run.status)
        time.sleep(0.5)
    return run

In [16]:
run_1 = wait_on_run(run,thread)

requires_action


In [18]:
print(run_1)

Run(id='run_MQvkJtkHdsfwa2N0Lv6SzScK', assistant_id='asst_AeWhKBH9uqoaHjJkVasyNITT', cancelled_at=None, completed_at=None, created_at=1715975960, expires_at=1715976560, failed_at=None, incomplete_details=None, instructions='Вы помогаете создать тест для проверки определений слов, предоставляя слово, а затем несколько вариантов определений, где только один вариант является правильным.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo-0125', object='thread.run', required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_5a6SKhp9IFHUjZzrQY7KepIR', function=Function(arguments='{"word":"abate","definitions_list":["to reduce in amount, degree, or intensity","to increase in amount, degree, or intensity","to stay the same in amount, degree, or intensity"]}', name='word_definition_quiz'), type='function')]), type='submit_tool_outputs'), response_format='auto', start

## Step 5: Take Run Results and Call Function Locally

In [20]:
import json

# Extract single tool call
tool_call = run_1.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: word_definition_quiz
Function Arguments:


{'word': 'abate',
 'definitions_list': ['to reduce in amount, degree, or intensity',
  'to increase in amount, degree, or intensity',
  'to stay the same in amount, degree, or intensity']}

In [21]:
answer = word_definition_quiz(arguments['word'],arguments['definitions_list'])

Привет! Давайте проверим ваши знания о словах.
Какое правильное определение этого слова abate



Определение #0 это: to reduce in amount, degree, or intensity


Определение #1 это: to increase in amount, degree, or intensity


Определение #2 это: to stay the same in amount, degree, or intensity




In [22]:
answer

'to increase in amount, degree, or intensity'

## Step 6: Return Local Function Response to the Tool in the Assistant

In [25]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(answer),
        }
    ],
)

BadRequestError: Error code: 400 - {'error': {'message': 'Runs in status "completed" do not accept tool outputs.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [26]:
run = wait_on_run(run, thread)

completed


In [27]:
run.status

'completed'

## Step 7: Retrieve Messages

In [28]:
messages = client.beta.threads.messages.list(thread.id)

In [29]:
for thread_message in messages:
    print(thread_message.content[0].text.value)
    print('\n')

Ответ неверный. Правильное определение слова "abate" - "to reduce in amount, degree, or intensity".


Создайте новый список слов и определений вопроса викторины. Затем сообщите мне, является ли полученный ответ правильным.




## Optional: Delete Assistant

In [39]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
response = client.beta.assistants.delete(my_assistants.data[0].id)
print(response)

AssistantDeleted(id='asst_7cVv7JptjJuNQWiA8mTTeDMb', deleted=True, object='assistant.deleted')
